In [6]:
from google.colab import files

uploaded = files.upload()  # 选择本地的 .parquet 文件上传


Saving yellow_tripdata_2024-01.parquet to yellow_tripdata_2024-01.parquet


In [4]:
!pip install pandas pyarrow

In [70]:
import pandas as pd


df = pd.read_parquet("yellow_tripdata_2024-01.parquet")

print(df.head(5))

print(df.info())


   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2024-01-01 00:57:55   2024-01-01 01:17:43              1.0   
1         1  2024-01-01 00:03:00   2024-01-01 00:09:36              1.0   
2         1  2024-01-01 00:17:06   2024-01-01 00:35:01              1.0   
3         1  2024-01-01 00:36:38   2024-01-01 00:44:56              1.0   
4         1  2024-01-01 00:46:51   2024-01-01 00:52:57              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           1.72         1.0                  N           186            79   
1           1.80         1.0                  N           140           236   
2           4.70         1.0                  N           236            79   
3           1.40         1.0                  N            79           211   
4           0.80         1.0                  N           211           148   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [71]:
loc = pd.read_csv("taxi_zone_lookup.csv")

print(loc .head(5))

   LocationID        Borough                     Zone service_zone
0           1            EWR           Newark Airport          EWR
1           2         Queens              Jamaica Bay    Boro Zone
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone
3           4      Manhattan            Alphabet City  Yellow Zone
4           5  Staten Island            Arden Heights    Boro Zone


In [72]:
# Optimize storage types to improve storage efficiency and save memory

df["passenger_count"] = df["passenger_count"].fillna(0).astype("int8")
df["RatecodeID"] = df["RatecodeID"].fillna(0).astype("int8")
df = df.astype({
    "VendorID": "int8",
    "payment_type": "int8",
    "store_and_fwd_flag": "category"
})
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int8          
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        int8          
 4   trip_distance          float64       
 5   RatecodeID             int8          
 6   store_and_fwd_flag     category      
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int8          
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [73]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.80,1,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.70,1,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.40,1,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.80,1,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [74]:
df[["tpep_pickup_datetime", "tpep_dropoff_datetime"]] = (
    df[["tpep_pickup_datetime", "tpep_dropoff_datetime"]].apply(lambda x: x.dt.floor("min")))
df_time = pd.concat([df["tpep_pickup_datetime"],df["tpep_dropoff_datetime"]]).dropna().drop_duplicates()
df_time_id = pd.DataFrame(df_time, columns=["datetime"])
df_time_id["time_id"] = (
    df_time_id["datetime"].dt.year.astype(int) * 100000000 +
    df_time_id["datetime"].dt.month.astype(int) * 1000000 +
    df_time_id["datetime"].dt.day.astype(int) * 10000 +
    df_time_id["datetime"].dt.hour.astype(int) * 100 +
    df_time_id["datetime"].dt.minute.astype(int)
).astype("int64")

df_time.head()

,0
0,2024-01-01 00:57:00
1,2024-01-01 00:03:00
2,2024-01-01 00:17:00
3,2024-01-01 00:36:00
4,2024-01-01 00:46:00


In [75]:

dim_time = pd.DataFrame({
    "time_id": df_time_id["time_id"],
    "datetime": df_time,
    "date": df_time.dt.date,
    "year": df_time.dt.year.astype("int16"),
    "month": df_time.dt.month.astype("int8"),
    "week": df_time.dt.isocalendar().week.astype("int8"),
    "day": df_time.dt.day.astype("int8"),
    "hour": df_time.dt.hour.astype("int8"),
    "weekday": df_time.dt.weekday.astype("int8")
}).reset_index(drop=True)


In [76]:
dim_time.head(10000)

,time_id,datetime,date,year,month,week,day,hour,weekday
0,202401010057,2024-01-01 00:57:00,2024-01-01,2024,1,1,1,0,0
1,202401010003,2024-01-01 00:03:00,2024-01-01,2024,1,1,1,0,0
2,202401010017,2024-01-01 00:17:00,2024-01-01,2024,1,1,1,0,0
3,202401010036,2024-01-01 00:36:00,2024-01-01,2024,1,1,1,0,0
4,202401010046,2024-01-01 00:46:00,2024-01-01,2024,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...
9995,202401072220,2024-01-07 22:20:00,2024-01-07,2024,1,1,7,22,6
9996,202401072230,2024-01-07 22:30:00,2024-01-07,2024,1,1,7,22,6
9997,202401072235,2024-01-07 22:35:00,2024-01-07,2024,1,1,7,22,6
9998,202401072257,2024-01-07 22:57:00,2024-01-07,2024,1,1,7,22,6


In [77]:
rate_code_mapping = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride"
}

dim_rate_codes = pd.DataFrame(list(rate_code_mapping.items()), columns=["rate_code_id", "rate_description"])
dim_rate_codes.head(6)

,rate_code_id,rate_description
0,1,Standard rate
1,2,JFK
2,3,Newark
3,4,Nassau or Westchester
4,5,Negotiated fare
5,6,Group ride


In [78]:
dim_location = loc.rename(columns={
    "LocationID": "location_id",
    "Borough": "borough",
    "Zone": "zone",
    "Service_zone": "service_zone"
    })
dim_location.head()

,location_id,borough,zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [79]:
payment_mapping = {
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip"
}
dim_payment_types = pd.DataFrame(list(payment_mapping.items()), columns=["payment_type", "payment_description"])
dim_payment_types.head(6)

,payment_type,payment_description
0,1,Credit card
1,2,Cash
2,3,No charge
3,4,Dispute
4,5,Unknown
5,6,Voided trip


In [80]:
vendors_mapping = {
    1: "Creative Mobile Technologies, LLC",
    2: "VeriFone Inc."
}
dim_vendors = pd.DataFrame(list(vendors_mapping.items()), columns=["vendor_id", "vendor_name"])
dim_vendors.head(6)

,vendor_id,vendor_name
0,1,"Creative Mobile Technologies, LLC"
1,2,VeriFone Inc.


In [81]:
df["pickup_time_id"] = (
    df["tpep_pickup_datetime"].dt.year * 10**8 +
    df["tpep_pickup_datetime"].dt.month * 10**6 +
    df["tpep_pickup_datetime"].dt.day * 10**4 +
    df["tpep_pickup_datetime"].dt.hour * 10**2 +
    df["tpep_pickup_datetime"].dt.minute
).astype("int64")
df["dropoff_time_id"] = (
    df["tpep_dropoff_datetime"].dt.year * 10**8 +
    df["tpep_dropoff_datetime"].dt.month * 10**6 +
    df["tpep_dropoff_datetime"].dt.day * 10**4 +
    df["tpep_dropoff_datetime"].dt.hour * 10**2 +
    df["tpep_dropoff_datetime"].dt.minute
).astype("int64")
df["pickup_time_id"].head()

,pickup_time_id
0,202401010057
1,202401010003
2,202401010017
3,202401010036
4,202401010046


In [82]:
#create stable trip_id

df["trip_id"] = (
    df["pickup_time_id"].astype(str) + "_" +
    df["VendorID"].astype(str) + "_" +
    df["PULocationID"].astype(str) + "_" +
    df["DOLocationID"].astype(str)
)

In [83]:
fact_trips = df[[
    "trip_id", "VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime",
    "PULocationID", "DOLocationID", "RatecodeID", "payment_type",
    "passenger_count", "trip_distance", "store_and_fwd_flag", "extra",
    "mta_tax", "improvement_surcharge", "congestion_surcharge",
    "tolls_amount", "fare_amount", "tip_amount", "total_amount", "Airport_fee"
]].rename(columns={
    "VendorID": "vendor_id",
    "tpep_pickup_datetime": "pickup_time_id",
    "tpep_dropoff_datetime": "dropoff_time_id",
    "PULocationID": "pickup_location_id",
    "DOLocationID": "dropoff_location_id",
    "payment_type": "payment_type_id",
    "RatecodeID": "rate_code_id"
    })

fact_trips.head()


,trip_id,vendor_id,pickup_time_id,dropoff_time_id,pickup_location_id,dropoff_location_id,rate_code_id,payment_type_id,passenger_count,trip_distance,store_and_fwd_flag,extra,mta_tax,improvement_surcharge,congestion_surcharge,tolls_amount,fare_amount,tip_amount,total_amount,Airport_fee
0,202401010057_2_186_79,2,2024-01-01 00:57:00,2024-01-01 01:17:00,186,79,1,2,1,1.72,N,1.0,0.5,1.0,2.5,0.0,17.7,0.00,22.70,0.0
1,202401010003_1_140_236,1,2024-01-01 00:03:00,2024-01-01 00:09:00,140,236,1,1,1,1.80,N,3.5,0.5,1.0,2.5,0.0,10.0,3.75,18.75,0.0
2,202401010017_1_236_79,1,2024-01-01 00:17:00,2024-01-01 00:35:00,236,79,1,1,1,4.70,N,3.5,0.5,1.0,2.5,0.0,23.3,3.00,31.30,0.0
3,202401010036_1_79_211,1,2024-01-01 00:36:00,2024-01-01 00:44:00,79,211,1,1,1,1.40,N,3.5,0.5,1.0,2.5,0.0,10.0,2.00,17.00,0.0
4,202401010046_1_211_148,1,2024-01-01 00:46:00,2024-01-01 00:52:00,211,148,1,1,1,0.80,N,3.5,0.5,1.0,2.5,0.0,7.9,3.20,16.10,0.0


In [ ]:

fact_trips.to_parquet("/content/fact_trips.parquet")
dim_time.to_parquet("/content/dim_time.parquet")
dim_location.to_parquet("/content/dim_location.parquet")
dim_rate_codes.to_parquet("/content/dim_rate_codes.parquet")
dim_payment_types.to_parquet("/content/dim_payment_types.parquet")
dim_vendors.to_parquet("/content/dim_vendors.parquet")
print("Parquet successful！✅")


In [ ]:
fact_trips.to_csv("/content/fact_trips.csv", index=False)
dim_time.to_csv("/content/dim_time.csv", index=False)
dim_location.to_csv("/content/dim_location.csv", index=False)
dim_rate_codes.to_csv("/content/dim_rate_code.csv", index=False)
dim_payment_types.to_csv("/content/dim_payment_types.csv", index=False)
dim_vendors.to_csv("/content/dim_vendors.csv")

print("csv successful！")
